In [15]:
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt

In [16]:
df = pd.DataFrame()

In [17]:
#merge data and set up original data
years = [2017, 2018, 2019, 2020, 2021]
df = pd.DataFrame()
for y in years:
    # might need to adjust file paths
    df_y =  pd.read_csv('Bitstamp_BTCUSD_{0}_minute.csv'.format(str(y)),)
    df_y = df_y.sort_values('date')
    df = df.append(df_y)

df = df.drop('symbol',axis = 1)
df['dates'] = pd.to_datetime(df['date']).dt.date
df['time'] = pd.to_datetime(df['date']).dt.time

#find missing dates
df_miss = pd.read_csv('BTC-USD.csv')
df_miss['Date'] = pd.to_datetime(df_miss['Date'])
df_miss.columns = ['date', 'open', 'high', 'low', 'close', 'adj close', 'Volume USD']
df_miss = df_miss.set_index('date')
df_miss['Volume BTC'] = df_miss['Volume USD'] / df_miss['close']
df_miss = df_miss[['open', 'close', 'high', 'low', 'Volume BTC', 'Volume USD']]

missing_dates = df_miss.index.tolist()

#get bigger dataset
bigdata = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
bigdata=bigdata.drop('Weighted_Price',axis=1)
bigdata['datetime'] = pd.to_datetime(bigdata['Timestamp'],unit='s')
bigdata['dates'] = pd.to_datetime(bigdata['datetime']).dt.date
bigdata['time'] = pd.to_datetime(bigdata['datetime']).dt.time

cols = ['Timestamp', 'datetime', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
        'Volume_(Currency)', 'dates', 'time']

bigdata_rearranged = bigdata[cols]

missing_dates_df = pd.DataFrame(missing_dates)
missing_dates_df['dates'] = pd.to_datetime(missing_dates_df[0]).dt.date

missingvals = bigdata_rearranged.loc[bigdata_rearranged['dates'].isin(missing_dates_df['dates'])]

missingvals = missingvals.rename(columns=dict(zip(missingvals.columns,bigdata_rearranged.columns)))

missingvals = missingvals.rename(columns=dict(zip(missingvals.columns,df.columns)))

df = df.append(missingvals)

df['date'] = pd.to_datetime(df['date'])

df = df.sort_values(by=['date'])

df.dropna(inplace=True)

#now lets get the actual technical features we want to use

df['SMA200'] = ta.trend.sma_indicator(close=df['close'],window=288000) #200 day simple moving average
df['SMA20'] = ta.trend.sma_indicator(close=df['close'],window=28800) #20 day simple moving average

df['next_min_change'] = ta.momentum.roc(df['close'],window=1).shift(-1) #minute over minute return

df['next_min_upordown'] = np.where(df['next_min_change']>0,1,0) #minute over minute up or down

#features related to volume
df['change_in_volume'] = df['Volume BTC']- df['Volume BTC'].shift(1) #simple absolute change
df['PVO'] = ta.momentum.PercentageVolumeOscillator(df['Volume BTC']).pvo_signal() #percentage volume oscillator, see ta docs
#df['PVO_positive'] = np.where(df['PVO']>0,1,0) # variable toget whether pvo was positive or negative. 
df['ADI'] = ta.volume.AccDistIndexIndicator(df['high'], df['low'], df['close'], df['Volume BTC']).acc_dist_index() #see ta docs
df['MFI'] = ta.volume.MFIIndicator(df['high'], df['low'], df['close'], df['Volume BTC'], 30).money_flow_index() #see ta docs
df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['Volume BTC']).on_balance_volume() #see ta docs

#momentum indicators
df['RSI'] = ta.momentum.RSIIndicator(df['close'], 30).rsi() #using last 30 minutes
df['TSI'] = ta.momentum.TSIIndicator(df['close'], 30, 15).tsi()

#crossover features

df['crossed_SMA200_breakdown']= np.where((df['open'] > df['SMA200']) & (df['close'] < df['SMA200']),1,0)#if price goes from above to below
df['crossed_SMA200_breakout'] = np.where((df['open'] < df['SMA200']) & (df['close'] > df['SMA200']),1,0)#if price goes from below to above





In [ ]:
dataset = df.set_index(df['date'])
dataset = dataset.dropna()
X = dataset.drop(['date','unix','open','high','low','close','Volume BTC','Volume USD','dates','time','next_min_change','next_min_upordown','SMA200', 'SMA20'],axis = 1)

y = dataset['next_min_upordown']



In [28]:
testdata = pd.read_csv('btc_test.csv',header=1)
testdata['dates'] = pd.to_datetime(testdata['date']).dt.date
testdata['time'] = pd.to_datetime(testdata['date']).dt.time

testdata = testdata.reindex(index=testdata.index[::-1])

#now lets get the actual technical features we want to use

testdata['SMA200'] = ta.trend.sma_indicator(close=testdata['close'],window=288000) #200 day simple moving average
testdata['SMA20'] = ta.trend.sma_indicator(close=testdata['close'],window=28800) #20 day simple moving average

testdata['next_min_change'] = ta.momentum.roc(testdata['close'],window=1).shift(-1) #minute over minute return

testdata['next_min_upordown'] = np.where(testdata['next_min_change']>0,1,0) #minute over minute up or down

#features related to volume
testdata['change_in_volume'] = testdata['Volume BTC']- testdata['Volume BTC'].shift(1) #simple absolute change
testdata['PVO'] = ta.momentum.PercentageVolumeOscillator(testdata['Volume BTC']).pvo_signal() #percentage volume oscillator, see ta docs
#testdata['PVO_positive'] = np.where(testdata['PVO']>0,1,0) # variable toget whether pvo was positive or negative. 
testdata['ADI'] = ta.volume.AccDistIndexIndicator(testdata['high'], testdata['low'], testdata['close'], testdata['Volume BTC']).acc_dist_index() #see ta docs
testdata['MFI'] = ta.volume.MFIIndicator(testdata['high'], testdata['low'], testdata['close'], testdata['Volume BTC'], 30).money_flow_index() #see ta docs
testdata['OBV'] = ta.volume.OnBalanceVolumeIndicator(testdata['close'], testdata['Volume BTC']).on_balance_volume() #see ta docs

#momentum indicators
testdata['RSI'] = ta.momentum.RSIIndicator(testdata['close'], 30).rsi() #using last 30 minutes
testdata['TSI'] = ta.momentum.TSIIndicator(testdata['close'], 30, 15).tsi()

#crossover features

testdata['crossed_SMA200_breakdown']= np.where((testdata['open'] > testdata['SMA200']) & (testdata['close'] < testdata['SMA200']),1,0)#if price goes from above to below
testdata['crossed_SMA200_breakout'] = np.where((testdata['open'] < testdata['SMA200']) & (testdata['close'] > testdata['SMA200']),1,0)#if price goes from below to above

dataset1 = testdata.set_index(testdata['date'])
dataset1 = dataset1.dropna()



dataset1 = dataset1.loc['2021-10-13 08:19:00':,:]

xTe= dataset1.drop(['date','unix','open','high','low','close','Volume BTC','Volume USD','dates','time','next_min_change','next_min_upordown','SMA200', 'SMA20','symbol'],axis = 1)

yTe = dataset1['next_min_upordown']

print(xTe.shape,yTe.shape)

(71871, 9) (71871,)


In [19]:
#separate into train and validation using sklearn
from sklearn.model_selection import TimeSeriesSplit


ts_cv = TimeSeriesSplit(n_splits=4)

all_splits = list(ts_cv.split(X, y))

In [20]:
#lets try gradient boosting since its an ensemble method that can handle categorical and numerical values

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV

gbclassifier = HistGradientBoostingClassifier()
scaler = MinMaxScaler()

pipeline = make_pipeline(scaler, gbclassifier)

def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring= ["accuracy"]
    )
    acc = cv_results["test_accuracy"]
    print(f"mean:     {acc.mean():.3f} +/- {acc.std():.3f}\n")
    return cv_results


defaulthgb = evaluate(pipeline, X, y, cv=ts_cv)



mean:     0.544 +/- 0.016



In [21]:
gb = gbclassifier.fit(X,y)

score = gb.train_score_
print(score)

[-0.69160581 -0.69058307 -0.68970181 -0.68897041 -0.68830588 -0.68774386
 -0.68725144 -0.68682237 -0.6864369  -0.68608035 -0.68575792 -0.68547524
 -0.68522779 -0.6850043  -0.6848073  -0.68461835 -0.68445669 -0.68428711
 -0.68415486 -0.68402416 -0.6839052  -0.68377512 -0.68366037 -0.68356659
 -0.68348402 -0.68339125 -0.68331892 -0.68324154 -0.68316824 -0.6831057
 -0.68303831 -0.68297837 -0.68291977 -0.68286465 -0.68281473 -0.68274154
 -0.68268935 -0.68264835 -0.68260877 -0.6825686  -0.68252159 -0.6824884
 -0.6824474  -0.68240787 -0.68237499 -0.68233258 -0.68227262 -0.68224038
 -0.68220739 -0.68217732 -0.68215069 -0.68212399 -0.68209685 -0.68207241
 -0.68205012 -0.68200317 -0.68197811 -0.68195368 -0.68193451 -0.68191359
 -0.68189114 -0.68186508 -0.6818451  -0.68182029 -0.6817961  -0.68175628
 -0.68173423 -0.68172045 -0.68170335 -0.6816819  -0.68165723 -0.68163325
 -0.68161813 -0.68158867 -0.68157319 -0.68155349 -0.68153469 -0.68151797
 -0.68150337 -0.68148628 -0.68146854 -0.68145478 -0.6

In [ ]:
#trying to tune model parameters


import warnings
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings("ignore")

gbclassifier = HistGradientBoostingClassifier()

parameters = {
    'max_iter': [1200],
    'learning_rate': [.05,0.1],
    'max_depth' : [25, 50, 75],
    'l2_regularization': [.5,1.5,2],
    'scoring': ['accuracy']
}
#start with gridsearch
hgb_grid = GridSearchCV(gbclassifier, parameters, n_jobs=5, cv=ts_cv, scoring='accuracy',refit=True)

hgb_grid.fit(X, y)

print(hgb_grid.best_params_)
# Print the best scores found
print()
print(hgb_grid.best_score_)


#THIS TAKES APROX 20-25 MINS TO RUN

In [34]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.model_selection import RandomizedSearchCV


gbclassifier = HistGradientBoostingClassifier()

parameters = {
    'max_iter': [1200],
    'learning_rate': np.linspace(0.01,.1,5),
    'max_depth' : [25, 50, 75],
    'l2_regularization': [.5,1.5,2],
    'scoring': ['accuracy'],
    'warm_start': True
    
}
#start with gridsearch
hgb_rand = RandomizedSearchCV(gbclassifier, parameters, n_jobs=5, cv=ts_cv, scoring='accuracy',refit=True)

hgb_rand.fit(X, y)

print(hgb_rand.best_params_)
# Print the best scores found
print()
print(hgb_rand.best_score_)


{'scoring': 'accuracy', 'max_iter': 1200, 'max_depth': 75, 'learning_rate': 0.0775, 'l2_regularization': 1.5}

0.5448555510625871


In [37]:
print('train acc: ', hgb_rand.score(X,y))
print('test acc: ', hgb_rand.score(xTe,yTe))

train acc:  0.556616794716269
test acc:  0.5133503081910645


In [55]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
parameters = {
    'n_estimators': np.linspace(200,1000,5,dtype=int),
    'max_depth' : [3,10,20],
    #'scoring': ['accuracy'],
    'warm_start': [True]
}

rfc_rand = RandomizedSearchCV(rfc, parameters, n_jobs=5, cv=ts_cv, scoring='accuracy',refit=True)
rfc_rand.fit(X,y)

In [54]:
print('train acc: ', rfc_rand.score(X,y))
print('test acc: ', rfc_rand.score(xTe,yTe))

train acc:  0.5392568630093904
test acc:  0.516146985571371
